# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys_copy import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


# ---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
%%capture --no-display

# Configure the map plot
map_city_data= city_data_df.hvplot.points(
    "Lng",
    "Lat",
     size = "Humidity",
     color= "City"
)

# Display the map
map_city_data

:Points   [Lng,Lat]   (City,Humidity)

# Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
#max temp lower than 27 but higher than 21
filtered_city_data_df = city_data_df.loc [(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
#wind speed less than 4.5m/s
filtered_city_data_df =filtered_city_data_df.loc [(city_data_df["Wind Speed"] < 4.5)]     
#zero cloudiness
filtered_city_data_df= filtered_city_data_df.loc [(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
filtered_city_data_df= filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


# Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,
573,tatarsk,RU,55.2190,75.9828,89,
574,canchungo,GW,12.0672,-16.0333,61,
575,iquique,CL,-20.2208,-70.1431,68,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "key": geoapify_key,
    "limit": 1 # just want first hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address =name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

asau - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
pevek - nearest hotel: No hotel found
husavik - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
zyryanka - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
baykit - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
meulaboh - nearest hotel: No hotel found
nakamura - nearest hotel: No hotel found
simao - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
kodinsk - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
glasgow - nearest hotel: No hotel found
ladario - nearest hotel: No hotel found
sao teotonio - nearest hotel: No hotel found
puri - nearest hotel: No hotel found
yumbe - nearest hotel: No hotel found
pangkalanbuun - nearest hotel: No hotel found
hay river - nearest hotel: No hotel found
verkhniye achaluki - nearest hot

carlyle - nearest hotel: No hotel found
kayerkan - nearest hotel: No hotel found
moron - nearest hotel: No hotel found
paamiut - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
seoul - nearest hotel: No hotel found
woodward - nearest hotel: No hotel found
georgiyevskoye - nearest hotel: No hotel found
mayo - nearest hotel: No hotel found
kysyl-syr - nearest hotel: No hotel found
tashara - nearest hotel: No hotel found
acajutla - nearest hotel: No hotel found
capoterra - nearest hotel: No hotel found
muisne - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
maniitsoq - nearest hotel: No hotel found
kalemie - nearest hotel: No hotel found
tagusao - nearest hotel: No hotel found
mao - nearest hotel: No hotel found
mahibadhoo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
panuco - nearest hotel: No hotel found
hervey bay - nearest hotel: No hotel found
fayaoue - nearest hotel: No hotel found
stolac - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No hotel found
3,jamestown,US,42.0970,-79.2353,77,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,No hotel found
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,No hotel found
573,tatarsk,RU,55.2190,75.9828,89,No hotel found
574,canchungo,GW,12.0672,-16.0333,61,No hotel found
575,iquique,CL,-20.2208,-70.1431,68,No hotel found


# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE